In [12]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
from sqlalchemy import create_engine, inspect, func ,distinct,inspect
import numpy as np
import pandas as pd
import datetime as dt
import datetime

# Reflect Tables into SQLAlchemy ORM

In [1]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
import datetime

In [2]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [3]:
# reflect an existing database into a new model
Base = automap_base()
Base.prepare(engine, reflect=True)
# reflect the tables

In [4]:
# We can view all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [5]:
# Save references to each table
measurement = Base.classes.measurement
station = Base.classes.station

In [6]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Climate Analysis

In [9]:
inspector = inspect(engine)
inspector.get_table_names()

['measurement', 'station']

### # Get a list of column names and types

In [10]:
columns = inspector.get_columns('measurement')
for c in columns:
    print(c['name'], c["type"])
# columns

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


# Design a query to retrieve the last 12 months of precipitation data and plot the results

In [17]:
from datetime import datetime
# Maximum date data available in database
max_date_in_db = session.query(func.max(measurement.date)).one()
max_date_in_db = datetime.strptime(max_date_in_db[0], '%Y-%m-%d')
one_year_before_maxdate = datetime(max_date_in_db.year -1, max_date_in_db.month, max_date_in_db.day)
query_result = session.query(measurement.id,measurement.station,measurement.date,
                        measurement.prcp,measurement.tobs).filter(measurement.date 
                                                                  >one_year_before_maxdate).filter(measurement.date 
                                                                  <= max_date_in_db).all()
for row in query_result:
    print(row)


(2365, 'USC00519397', '2016-08-24', 0.08, 79.0)
(2366, 'USC00519397', '2016-08-25', 0.08, 80.0)
(2367, 'USC00519397', '2016-08-26', 0.0, 79.0)
(2368, 'USC00519397', '2016-08-27', 0.0, 77.0)
(2369, 'USC00519397', '2016-08-28', 0.01, 78.0)
(2370, 'USC00519397', '2016-08-29', 0.0, 78.0)
(2371, 'USC00519397', '2016-08-30', 0.0, 79.0)
(2372, 'USC00519397', '2016-08-31', 0.13, 80.0)
(2373, 'USC00519397', '2016-09-01', 0.0, 81.0)
(2374, 'USC00519397', '2016-09-02', 0.0, 80.0)
(2375, 'USC00519397', '2016-09-03', 0.0, 79.0)
(2376, 'USC00519397', '2016-09-04', 0.03, 75.0)
(2377, 'USC00519397', '2016-09-05', None, 79.0)
(2378, 'USC00519397', '2016-09-06', None, 76.0)
(2379, 'USC00519397', '2016-09-07', 0.05, 76.0)
(2380, 'USC00519397', '2016-09-08', 0.0, 80.0)
(2381, 'USC00519397', '2016-09-09', 0.03, 79.0)
(2382, 'USC00519397', '2016-09-10', 0.0, 78.0)
(2383, 'USC00519397', '2016-09-11', 0.05, 76.0)
(2384, 'USC00519397', '2016-09-12', 0.0, 78.0)
(2385, 'USC00519397', '2016-09-13', 0.02, 78.0)
(2

### Calculate the date 1 year ago from the last data point in the database

In [23]:
max_date_in_db = session.query(func.max(measurement.date)).one()
max_date_in_db = datetime.strptime(max_date_in_db[0], '%Y-%m-%d')

last_year_date = datetime(max_date_in_db.year -1, max_date_in_db.month, max_date_in_db.day)
print(f'Max Date in database: {max_date_in_db}, Last Year Date:{last_year_date}')

Max Date in database: 2017-08-23 00:00:00, Last Year Date:2016-08-23 00:00:00


### Perform a query to retrieve the data and precipitation scores

In [27]:
retrieve_query_result = session.query(measurement.date,
                        measurement.prcp).filter(measurement.date 
                                                                  >one_year_before_maxdate).filter(measurement.date 
                                                                  <= max_date_in_db).all()
for row in retrieve_query_result:
    print(row)

('2016-08-24', 0.08)
('2016-08-25', 0.08)
('2016-08-26', 0.0)
('2016-08-27', 0.0)
('2016-08-28', 0.01)
('2016-08-29', 0.0)
('2016-08-30', 0.0)
('2016-08-31', 0.13)
('2016-09-01', 0.0)
('2016-09-02', 0.0)
('2016-09-03', 0.0)
('2016-09-04', 0.03)
('2016-09-05', None)
('2016-09-06', None)
('2016-09-07', 0.05)
('2016-09-08', 0.0)
('2016-09-09', 0.03)
('2016-09-10', 0.0)
('2016-09-11', 0.05)
('2016-09-12', 0.0)
('2016-09-13', 0.02)
('2016-09-14', 1.32)
('2016-09-15', 0.42)
('2016-09-16', 0.06)
('2016-09-17', 0.05)
('2016-09-18', 0.0)
('2016-09-19', 0.0)
('2016-09-20', 0.0)
('2016-09-21', 0.0)
('2016-09-22', 0.02)
('2016-09-23', 0.0)
('2016-09-24', 0.0)
('2016-09-25', 0.0)
('2016-09-26', 0.06)
('2016-09-27', 0.02)
('2016-09-28', 0.0)
('2016-09-29', 0.0)
('2016-09-30', 0.0)
('2016-10-01', 0.0)
('2016-10-02', 0.0)
('2016-10-03', 0.0)
('2016-10-04', 0.0)
('2016-10-05', 0.0)
('2016-10-06', 0.0)
('2016-10-07', 0.0)
('2016-10-08', 0.0)
('2016-10-09', 0.0)
('2016-10-10', 0.0)
('2016-10-11', 0.0)
('

### Save the query results as a Pandas DataFrame and set the index to the date column

In [28]:
df_query = pd.DataFrame(query_result)
df_query_date_idx = df_query.set_index('date')
df_query_date_idx.head(5)

,id,station,prcp,tobs
date,,,,
2016-08-24,2365,USC00519397,0.08,79.0
2016-08-25,2366,USC00519397,0.08,80.0
2016-08-26,2367,USC00519397,0.00,79.0
2016-08-27,2368,USC00519397,0.00,77.0
2016-08-28,2369,USC00519397,0.01,78.0


### Sort the dataframe by date

In [29]:
df_query_date_idx.sort_index().head(5)

,id,station,prcp,tobs
date,,,,
2016-08-24,2365,USC00519397,0.08,79.0
2016-08-24,8826,USC00517948,NaN,78.0
2016-08-24,14609,USC00519281,2.15,77.0
2016-08-24,7332,USC00514830,2.28,80.0
2016-08-24,5092,USC00513117,2.15,76.0


### Use Pandas Plotting with Matplotlib to plot the data

### Use Pandas to calcualte the summary statistics for the precipitation data

In [30]:
df_query_date_idx["prcp"].describe()

count    2015.000000
mean        0.176462
std         0.460288
min         0.000000
25%         0.000000
50%         0.020000
75%         0.130000
max         6.700000
Name: prcp, dtype: float64

### Design a query to show how many stations are available in this dataset?

In [ ]:
distinct_count_station_in_dataset= session.query(func.count(distinct(measurement.station))).all()
print(f'Number of Unique stations available in DataSet 12 months record : {distinct_count_station_in_dataset[0]}')

### What are the most active stations? (i.e. what stations have the most rows)?
##### List the stations and the counts in descending order.


In [31]:
sel = [measurement.station,func.count(measurement.station)]
active_stations = session.query(*sel).group_by(measurement.station).\
                  order_by(func.count(measurement.station).desc()).all()
active_stations

[('USC00519281', 2772),
 ('USC00519397', 2724),
 ('USC00513117', 2709),
 ('USC00519523', 2669),
 ('USC00516128', 2612),
 ('USC00514830', 2202),
 ('USC00511918', 1979),
 ('USC00517948', 1372),
 ('USC00518838', 511)]

### Using the station id from the previous query, calculate the lowest temperature recorded, 
### highest temperature recorded, and average temperature of the most active station?


In [54]:
#inspector.get_columns('measurement')

In [53]:
most_active_station_id = active_stations[0].station
sel = [measurement.station,func.min(measurement.tobs),func.max(measurement.tobs),func.avg(measurement.tobs)]
most_active_station_temp = session.query(*sel).filter(measurement.station==most_active_station_id)
for row in most_active_station_temp:
    print(row)


('USC00519281', 54.0, 85.0, 71.66378066378067)


### Choose the station with the highest number of temperature observations.
####     Query the last 12 months of temperature observation data for this station and plot the results as a histogram

In [60]:
sel = [measurement.station,func.count(measurement.tobs)]
most_number_of_tobs_station = session.query(*sel).filter \
                              (measurement.date >one_year_before_maxdate).filter\
                              (measurement.date <= max_date_in_db).group_by(measurement.station).\
                              order_by(func.count(measurement.tobs).desc()).first()
most_number_of_tobs_station

('USC00519397', 360)

## Bonus Challenge Assignment

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.


In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)


In [ ]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation



In [ ]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
daily_normals("01-01")

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip

# Use the start and end date to create a range of dates

# Stip off the year and save a list of %m-%d strings

# Loop through the list of %m-%d strings and calculate the normals for each date


In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index


In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
